In [155]:
# %pip install langchain-community tiktoken langchainhub faiss-cpu langchain langgraph langchain-text-splitters beautifulsoup4 sentence-transformers langchain-huggingface transformers accelerate torch 


### Import des Librairies

In [2]:
import getpass
import os
import pandas as pd
import time


### Récupération des données

In [5]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="C:/Users/peggy/Documents/Fabeon/deep_learning/AAPL.csv", encoding = 'utf-8')

data = loader.load()

print(data[:10])

[Document(metadata={'source': 'C:/Users/peggy/Documents/Fabeon/deep_learning/AAPL.csv', 'row': 0}, page_content='Date: 2023-12-16 22:00:00 UTC\nArticle_title: My 6 Largest Portfolio Holdings Heading Into 2024 -- and the Important Investing Lesson I Learned From Each One\nStock_symbol: AAPL\nUrl: https://www.nasdaq.com/articles/my-6-largest-portfolio-holdings-heading-into-2024-and-the-important-investing-lesson-i\nPublisher: \nAuthor: \nArticle: After an absolute disaster of a year in 2022, the stock market appears to have turned the corner. Each of the major market indexes has gained more than 20% from their respective trough. Perhaps more importantly, the S&P 500 and the Nasdaq Composite are within striking distance of new highs, which will check the final box marking the start of a new bull market.\nClosing out the old and ringing in the new is a great time for examination, and one of the places I start is with my portfolio. A review of my top investments and how they came to be that

In [6]:
df = pd.read_csv('C:/Users/peggy/Documents/Fabeon/deep_learning/AAPL.csv', sep=',', encoding='utf-8') 
df


,Date,Article_title,Stock_symbol,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,2023-12-16 22:00:00 UTC,My 6 Largest Portfolio Holdings Heading Into 2...,AAPL,https://www.nasdaq.com/articles/my-6-largest-p...,NaN,NaN,"After an absolute disaster of a year in 2022, ...",3: Apple There's little question that Apple (N...,3: Apple There's little question that Apple (N...,3: Apple There's little question that Apple (N...,3: Apple There's little question that Apple (N...
1,2023-12-16 22:00:00 UTC,Brokers Suggest Investing in Apple (AAPL): Rea...,AAPL,https://www.nasdaq.com/articles/brokers-sugges...,NaN,NaN,"When deciding whether to buy, sell, or hold a ...",Let's take a look at what these Wall Street he...,Click to get this free report Apple Inc. (AAPL...,Let's take a look at what these Wall Street he...,Brokerage Recommendation Trends for AAPL Let's...
2,2023-12-16 21:00:00 UTC,"Company News for Dec 19, 2023",AAPL,https://www.nasdaq.com/articles/company-news-f...,NaN,NaN,Shares of Apple Inc. AAPL lost 0.9% on China’s...,Shares of Apple Inc. AAPL lost 0.9% on China’s...,Click to get this free report Apple Inc. (AAPL...,Click to get this free report Apple Inc. (AAPL...,Click to get this free report Apple Inc. (AAPL...
3,2023-12-16 21:00:00 UTC,NVIDIA (NVDA) Up 243% YTD: Will It Carry Momen...,AAPL,https://www.nasdaq.com/articles/nvidia-nvda-up...,NaN,NaN,NVIDIA Corporation NVDA has witnessed a remark...,Other Stocks in the $1T Club Apart from NVIDIA...,Other Stocks in the $1T Club Apart from NVIDIA...,Other Stocks in the $1T Club Apart from NVIDIA...,Other Stocks in the $1T Club Apart from NVIDIA...
4,2023-12-16 21:00:00 UTC,"Pre-Market Most Active for Dec 19, 2023 : BMY,...",AAPL,https://www.nasdaq.com/articles/pre-market-mos...,NaN,NaN,The NASDAQ 100 Pre-Market Indicator is up 10.1...,"Apple Inc. (AAPL) is +0.86 at $196.75, with 1,...","Apple Inc. (AAPL) is +0.86 at $196.75, with 1,...","Apple Inc. (AAPL) is +0.86 at $196.75, with 1,...","Apple Inc. (AAPL) is +0.86 at $196.75, with 1,..."
...,...,...,...,...,...,...,...,...,...,...,...
9333,2020-05-28 00:00:00 UTC,"The Zacks Analyst Blog Highlights: Apple, Exxo...",AAPL,http://www.zacks.com/stock/news/950055/the-zac...,Zacks,NaN,NaN,NaN,NaN,NaN,NaN
9334,2020-05-28 00:00:00 UTC,Growth Stocks for 2020: Trading Tech Stocks + ...,AAPL,https://talkmarkets.com/content/growth-stocks-...,TalkMarkets,NaN,NaN,NaN,NaN,NaN,NaN
9335,2020-05-28 00:00:00 UTC,Stocks Are Struggling To Post Gains On May 28 ...,AAPL,https://talkmarkets.com/content/stocks-are-str...,TalkMarkets,NaN,NaN,NaN,NaN,NaN,NaN
9336,2020-05-28 00:00:00 UTC,Swiss National Bank Ready To Buy Much More Tec...,AAPL,https://talkmarkets.com/content/swiss-national...,TalkMarkets,NaN,NaN,NaN,NaN,NaN,NaN


### Embedding

In [10]:
list_url = df['Url'].head(10).tolist()
list_url 

['https://www.nasdaq.com/articles/my-6-largest-portfolio-holdings-heading-into-2024-and-the-important-investing-lesson-i',
 'https://www.nasdaq.com/articles/brokers-suggest-investing-in-apple-aapl%3A-read-this-before-placing-a-bet',
 'https://www.nasdaq.com/articles/company-news-for-dec-19-2023',
 'https://www.nasdaq.com/articles/nvidia-nvda-up-243-ytd%3A-will-it-carry-momentum-in-2024',
 'https://www.nasdaq.com/articles/pre-market-most-active-for-dec-19-2023-%3A-bmy-sqqq-nio-ubs-tqqq-uber-nvda-aapl-gotu-can',
 'https://www.nasdaq.com/articles/3-artificial-intelligence-ai-stocks-for-2024-and-beyond',
 'https://www.nasdaq.com/articles/aapl-quantitative-stock-analysis-10',
 'https://www.nasdaq.com/articles/should-vanguard-sp-500-etf-voo-be-on-your-investing-radar-11',
 'https://www.nasdaq.com/articles/is-flexshares-quality-dividend-etf-qdf-a-strong-etf-right-now-0',
 'https://www.nasdaq.com/articles/is-flexshares-stoxx-us-esg-select-index-fund-esg-a-strong-etf-right-now-0']

In [12]:
# Choix d'un modèle 
from langchain_huggingface import HuggingFaceEmbeddings
 
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [13]:
# from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# réduction du nombre de lignes
df_sample = df.head(1000)

# Convertir les lignes du DataFrame en objets Document
docs = [
    Document(page_content=row["Article"])  
    for _, row in df_sample.iterrows()
    if isinstance(row["Article"], str) and row["Article"].strip() != ""
]

# Découpage du texte
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50                         # chaque morceaux contiendra 100 tokens maxi  # chaque chunk chevauche le précédent de 50 tokens pour conserver le contexte
)
doc_splits = text_splitter.split_documents(docs)             # liste de textes prêts à être vectorisés

# Création de la BDD vectorielle
vectorstore = FAISS.from_documents(
    documents=doc_splits,                                    # convertit chaque chunk de texte en vecteurs
    embedding=embeddings                                     # via le modèle d'embedding
)
retriever = vectorstore.as_retriever()                       # transforme la base FAISS en retriever utilisable par un agent longChain

In [14]:
# Création de l'outil de récupération (utilisable par un agent LangChain)
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,               # objet FAISS
    "retrieve_blog_posts",   # nom interne de l'outil utilisé par l'agent qd il planifie des actions
    "Search and return information about Lilian Weng blog posts on LLM agents, prompt engineering, and adversarial attacks on LLMs.",
)

tools = [retriever_tool]  # l'outil est mis ds une liste

In [15]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

# Classe qui représente l'état courant d'un agent dans un graphe LangGraph
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]   # ajout d'un nouveau message à la liste

### Génération de texte

In [153]:
# pip install ollama


In [154]:
# pip install -U langchain-ollama


In [17]:
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from typing import Literal

llm = OllamaLLM(model="mistral")
output_parser = StrOutputParser()

def grade_documents(state) -> Literal["generate", "rewrite"]:
    """
    Vérifie si un document récupéré est pertinent par rapport à la question de l'utilisateur.

    Cette fonction utilise un modèle de langage (LLM) pour évaluer la pertinence sémantique
    entre une question et un document. Si le document est jugé pertinent, elle retourne "generate",
    sinon elle retourne "rewrite" pour inviter l'utilisateur à reformuler sa question.

    Args:
        state (dict): État courant de l'agent, contenant la liste des messages échangés.

    Returns:
        Literal["generate", "rewrite"]: Décision à suivre dans le graphe, selon la pertinence détectée.
    """
    print("---ÉVALUATION DE LA PERTINENCE---")

    # Définition du prompt pour le LLM
    prompt = PromptTemplate(
        template=(
            "Tu es un évaluateur chargé de déterminer la pertinence d'un document par rapport à une question utilisateur.\n\n"
            "Document :\n{context}\n\n"
            "Question :\n{question}\n\n"
            "Si le document contient des mots-clés ou un sens apparenté à la question, réponds 'yes', sinon réponds 'no'."
        ),
        input_variables=["context", "question"]
    )

    # Récupération des messages
    messages = state["messages"]
    question = messages[0].content     # Question utilisateur
    context = messages[-1].content     # Document récupéré

    # Création de la chaîne LLM avec parsing de la réponse
    chain = prompt | llm | output_parser
    result = chain.invoke({"context": context, "question": question}).strip().lower()

    # Décision en fonction de la réponse du modèle
    if "yes" in result:
        print("---DÉCISION : DOCUMENT PERTINENT---")
        return {"next": "generate"}
    else:
        print("---DÉCISION : DOCUMENT NON PERTINENT---")
        return {"next": "rewrite"}

In [18]:
from langchain_core.messages import AIMessage

def agent(state: AgentState) -> AgentState:
    """
    Appelle le modèle via Ollama et ajoute la réponse dans l'état.

    Args:
        state (AgentState): L'état actuel (liste des messages)
    
    Returns:
        AgentState: L'état mis à jour avec la réponse du modèle
    """
    print("---CALL AGENT---")
    messages = state["messages"]
    last_user_message = messages[-1].content

    response = llm.invoke(last_user_message)

    return {"messages": [AIMessage(content=response)]}


In [19]:
# Node 
def generate(state: AgentState) -> AgentState:
    """
    Utilise le modèle de langage (LLM) pour générer une réponse à partir du dernier
    message utilisateur contenu dans l'état.

    Args:
        state (AgentState): L'état actuel, contenant l'historique des messages.

    Returns:
        AgentState: L'état mis à jour avec la réponse générée par le modèle.
    """
    print("---GENERATE---")
    messages = state["messages"]
    user_message = messages[-1]  # Dernier message utilisateur
    response = llm.invoke(user_message.content)
    return {"messages": [AIMessage(content=response)]}


In [20]:
# Node
def rewrite(state: AgentState) -> AgentState:
    """
    Répond à l'utilisateur lorsque le document récupéré n'est pas jugé pertinent par le système.
    Invite l'utilisateur à reformuler sa question pour améliorer la recherche.

    Args:
        state (AgentState): L'état actuel, contenant l'historique des messages.

    Returns:
        AgentState: L'état mis à jour avec un message demandant une reformulation.
    """
    print("---REWRITE---")
    return {"messages": [AIMessage(content="Le document n'est pas pertinent. Pouvez-vous reformuler ?")]}


In [21]:
from langgraph.graph import StateGraph, END

In [22]:
# Construction du graphe LangGraph
builder = StateGraph(AgentState)                           # Spécifie le type d'état partagé entre les nodes

builder.add_node("grade_documents", grade_documents)       # Évaluation de la pertinence
builder.add_node("generate", generate)                     # Génération de la réponse
builder.add_node("rewrite", rewrite)                       # Reformulation si non pertinent
builder.add_node("agent", agent)                           # Appel final à l'agent principal

In [23]:
# Logique de décision
builder.set_entry_point("grade_documents")                 # Point d’entrée du graphe (premier node exécuté)
builder.add_conditional_edges(
    "grade_documents",
    lambda state: state["next"],  
    {
        "generate": "generate",   
        "rewrite": "rewrite"
    }
)

In [24]:
# Fin du graphe
builder.add_edge("agent", END)                             # Fin du graphe après l’agent          
builder.add_edge("rewrite", END)

### Execution du graphe

In [26]:
# Compilation du graphe pour l'exécuter
graph = builder.compile()

In [27]:
import pprint

# Question
inputs = {
    "messages": [
        ("user", "Quelles sont les actions les plus performantes en ce moment ?"),
    ]  # answer = "Zacks"
}
for output in graph.stream(inputs):
    for key, value in output.items():
        pprint.pprint(f"Output from node '{key}':")
        pprint.pprint(value, indent=2, width=80, depth=None)
    print("\n\n")

---ÉVALUATION DE LA PERTINENCE---
---DÉCISION : DOCUMENT PERTINENT---
"Output from node 'grade_documents':"
None



---GENERATE---
"Output from node 'generate':"
{ 'messages': [ AIMessage(content=" Il est difficile de déterminer quels sont les meilleurs investissements ou les meilleures actions aujourd'hui car cela dépend de beaucoup de facteurs, tels que la régulation financière, l'économie politique et les tendances générales du marché.\n\nToutefois, voici quelques-unes des sociétés qui sont souvent mentionnées comme des investissements prometteurs :\n\n1. Microsoft Corporation (MSFT) : La société est un leader mondial dans le domaine de l'informatique et des technologies. Elle a vu son activité croître significativement pendant la pandémie du Covid-19 en raison d'une demande accrue en services de cloud computing, de collaboration et de sécurité.\n2. Amazon.com Inc. (AMZN) : L'entreprise est un leader mondial dans le commerce en ligne et les services de cloud computing. Le marché en 

### Optimisation

In [28]:
# Combinaison chunk_size et overlap
chunk_sizes = [500, 1000, 1500]
overlaps = [100, 200]

for chunk_size in chunk_sizes:
    for overlap in overlaps:
        print(f"➡️ Test avec chunk_size = {chunk_size}, overlap = {overlap}")
        text_splitter2 = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
        doc_splits2 = text_splitter2.split_documents(docs)
        print(f"📦 {len(doc_splits2)} chunks générés")

        vectorstore2 = FAISS.from_documents(doc_splits2, embedding=embeddings)
        retriever2 = vectorstore2.as_retriever()
        
        # Test d’une requête
        print("🔍 Résultat de la requête test:")
        test_question = "Quelles sont les actions les plus performantes en ce moment "
        results2 = retriever2.get_relevant_documents(test_question, k=1)   # la question est encodée en vecteur avec le même embedding
        print(results2[0].page_content)

➡️ Test avec chunk_size = 500, overlap = 100
📦 12817 chunks générés
🔍 Résultat de la requête test:
market making or asset management activities of any securities. These returns are from hypothetical portfolios consisting of stocks with Zacks Rank = 1 that were rebalanced monthly with zero transaction costs. These are not the returns of actual portfolios of stocks. The S&P 500 is an unmanaged index. Visit https://www.zacks.com/performance for information about the performance numbers displayed in this press release.
➡️ Test avec chunk_size = 500, overlap = 200
📦 13735 chunks générés


C:\Users\peggy\AppData\Local\Temp\ipykernel_20580\3070219076.py:18: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results2 = retriever2.get_relevant_documents(test_question, k=1)   # la question est encodée en vecteur avec le même embedding


🔍 Résultat de la requête test:
Moreover, momentum stocks serve as a strategic tool for portfolio diversification. They offer exposure to various sectors and industries, potentially mitigating overall risk. The approach taps into the psychological aspect of investing, leveraging the human tendency to ‘follow the herd.’
➡️ Test avec chunk_size = 1000, overlap = 100
📦 5873 chunks générés
🔍 Résultat de la requête test:
FUNDAMENTAL MOMENTUM: PASS
TWELVE MINUS ONE MOMENTUM: PASS
FINAL RANK: PASS
➡️ Test avec chunk_size = 1000, overlap = 200
📦 6065 chunks générés
🔍 Résultat de la requête test:
FUNDAMENTAL MOMENTUM: PASS
TWELVE MINUS ONE MOMENTUM: PASS
FINAL RANK: PASS
➡️ Test avec chunk_size = 1500, overlap = 100
📦 3894 chunks générés
🔍 Résultat de la requête test:
Volatility is back down at very low levels, with the VIX Index closing at 12.63 yesterday. When volatility is low, options become cheaper, so today we’re taking a look at the Long Straddle Screener.
A long straddle is an advanced o

In [31]:
# Combinaison chunk_size et overlap
chunk_sizes = [200, 300, 500]
overlaps = [50, 100]

for chunk_size in chunk_sizes:
    for overlap in overlaps:
        print(f"➡️ Test avec chunk_size = {chunk_size}, overlap = {overlap}")
        text_splitter3 = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
        doc_splits3 = text_splitter3.split_documents(docs)
        print(f"📦 {len(doc_splits3)} chunks générés")

        vectorstore3 = FAISS.from_documents(doc_splits3, embedding=embeddings)
        retriever3 = vectorstore3.as_retriever()
        
        # Test d’une requête
        print("🔍 Résultat de la requête test:")
        test_question = "Quelles sont les actions les plus performantes en ce moment ?"
        results3 = retriever3.get_relevant_documents(test_question, k=1)   # la question est encodée en vecteur avec le même embedding
        print(results3[0].page_content[:300], "...\n")


➡️ Test avec chunk_size = 200, overlap = 50
📦 34153 chunks générés
🔍 Résultat de la requête test:
2. MercadoLibre ...

➡️ Test avec chunk_size = 200, overlap = 100
📦 38333 chunks générés
🔍 Résultat de la requête test:
2. MercadoLibre ...

➡️ Test avec chunk_size = 300, overlap = 50
📦 22270 chunks générés
🔍 Résultat de la requête test:
FUNDAMENTAL MOMENTUM: PASS
TWELVE MINUS ONE MOMENTUM: PASS
FINAL RANK: PASS ...

➡️ Test avec chunk_size = 300, overlap = 100
📦 22970 chunks générés
🔍 Résultat de la requête test:
FUNDAMENTAL MOMENTUM: PASS
TWELVE MINUS ONE MOMENTUM: PASS
FINAL RANK: PASS ...

➡️ Test avec chunk_size = 500, overlap = 50
📦 12671 chunks générés
🔍 Résultat de la requête test:
The following table summarizes whether the stock meets each of this strategy's tests. Not all criteria in the below table receive equal weighting or are independent, but the table provides a brief overview of the strong and weak points of the security in the context of the strategy's criteria.
FUNDA ...

### Similarité

In [33]:
# On augmente à k=3  qui permet de récupérer plus de contexte pour l'agent génératif.
chunk_sizes = [500, 1000, 1500]
overlaps = [100, 200]

for chunk_size in chunk_sizes:
    for overlap in overlaps:
        print(f"➡️ Test avec chunk_size = {chunk_size}, overlap = {overlap}")
        text_splitter4 = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
        doc_splits4 = text_splitter4.split_documents(docs)
        print(f"📦 {len(doc_splits4)} chunks générés")

        vectorstore4 = FAISS.from_documents(doc_splits4, embedding=embeddings)
        retriever4 = vectorstore4.as_retriever()
        
        # Test d’une requête
        print("🔍 Résultat de la requête test:")
        test_question = "Quelles sont les actions les plus performantes en ce moment ?"
        results4 = retriever4.get_relevant_documents(test_question, k=3)   # la question est encodée en vecteur avec le même embedding
        print(results4[0].page_content)

➡️ Test avec chunk_size = 500, overlap = 100
📦 12817 chunks générés
🔍 Résultat de la requête test:
market making or asset management activities of any securities. These returns are from hypothetical portfolios consisting of stocks with Zacks Rank = 1 that were rebalanced monthly with zero transaction costs. These are not the returns of actual portfolios of stocks. The S&P 500 is an unmanaged index. Visit https://www.zacks.com/performance for information about the performance numbers displayed in this press release.
➡️ Test avec chunk_size = 500, overlap = 200
📦 13735 chunks générés
🔍 Résultat de la requête test:
The following table summarizes whether the stock meets each of this strategy's tests. Not all criteria in the below table receive equal weighting or are independent, but the table provides a brief overview of the strong and weak points of the security in the context of the strategy's criteria.
FUNDAMENTAL MOMENTUM: PASS
TWELVE MINUS ONE MOMENTUM: PASS
FINAL RANK: PASS
➡️ Test a

In [44]:
# k=3 et chunk_sizes plus petits 
chunk_sizes = [100, 200, 300]
overlaps = [50, 100]

for chunk_size in chunk_sizes:
    for overlap in overlaps:
        print(f"➡️ Test avec chunk_size = {chunk_size}, overlap = {overlap}")
        text_splitter5 = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
        doc_splits5 = text_splitter5.split_documents(docs)
        print(f"📦 {len(doc_splits5)} chunks générés")

        vectorstore5 = FAISS.from_documents(doc_splits5, embedding=embeddings)
        retriever5 = vectorstore5.as_retriever()
        
        # Test d’une requête
        print("🔍 Résultat de la requête test:")
        test_question = "Quelles sont les actions les plus performantes en ce moment ?"
        results5 = retriever5.get_relevant_documents(test_question, k=3)   # la question est encodée en vecteur avec le même embedding
        print(results5[0].page_content)

➡️ Test avec chunk_size = 100, overlap = 50
📦 81817 chunks générés
🔍 Résultat de la requête test:
2. MercadoLibre
➡️ Test avec chunk_size = 100, overlap = 100
📦 341687 chunks générés
🔍 Résultat de la requête test:
2. MercadoLibre
➡️ Test avec chunk_size = 200, overlap = 50
📦 34153 chunks générés
🔍 Résultat de la requête test:
2. MercadoLibre
➡️ Test avec chunk_size = 200, overlap = 100
📦 38333 chunks générés
🔍 Résultat de la requête test:
2. MercadoLibre
➡️ Test avec chunk_size = 300, overlap = 50
📦 22270 chunks générés
🔍 Résultat de la requête test:
FUNDAMENTAL MOMENTUM: PASS
TWELVE MINUS ONE MOMENTUM: PASS
FINAL RANK: PASS
➡️ Test avec chunk_size = 300, overlap = 100
📦 22970 chunks générés
🔍 Résultat de la requête test:
FUNDAMENTAL MOMENTUM: PASS
TWELVE MINUS ONE MOMENTUM: PASS
FINAL RANK: PASS


### distance_strategy

Le paramètre distance_strategy dans le contexte de FAISS permet de spécifier la stratégie utilisée pour calculer la similarité entre les vecteurs. 
Il définit la manière dont les distances entre les vecteurs seront mesurées.

In [35]:
# Test avec distance Euclidienne
vectorstore_l2 = FAISS.from_documents(doc_splits, embedding=embeddings, distance_strategy="l2")
retriever_l2 = vectorstore_l2.as_retriever()
        
# Test d’une requête
print("🔍 Résultat de la requête test:")
test_question = "Quelles sont les actions les plus performantes en ce moment ?"
results_l2 = retriever_l2.get_relevant_documents(test_question, k=1)   # la question est encodée en vecteur avec le même embedding
print(results_l2[0].page_content)

🔍 Résultat de la requête test:
Moreover, momentum stocks serve as a strategic tool for portfolio diversification. They offer exposure to various sectors and industries, potentially mitigating overall risk. The approach taps into the psychological aspect of investing, leveraging the human tendency to ‘follow the herd.’
With that said, these selections stand out for their robust performance and upward trajectories. They also inspire confidence among those who scrutinize the market’s every pulse. Consequently, each stock in this list represents a unique opportunity.


In [37]:
# Test avec distance Cosinus
vectorstore_cos = FAISS.from_documents(doc_splits, embedding=embeddings, distance_strategy="cosine")
retriever_cos = vectorstore_cos.as_retriever()
        
# Test d’une requête
print("🔍 Résultat de la requête test:")
test_question = "Quelles sont les actions les plus performantes en ce moment ?"
results_cos = retriever_cos.get_relevant_documents(test_question, k=1)   # la question est encodée en vecteur avec le même embedding
print(results_cos[0].page_content)

🔍 Résultat de la requête test:
Moreover, momentum stocks serve as a strategic tool for portfolio diversification. They offer exposure to various sectors and industries, potentially mitigating overall risk. The approach taps into the psychological aspect of investing, leveraging the human tendency to ‘follow the herd.’
With that said, these selections stand out for their robust performance and upward trajectories. They also inspire confidence among those who scrutinize the market’s every pulse. Consequently, each stock in this list represents a unique opportunity.


In [39]:
# Test avec le produit scalaire maximum
vectorstore_inner = FAISS.from_documents(doc_splits, embedding=embeddings, distance_strategy="MAX_INNER_PRODUCT")
retriever_inner = vectorstore_inner.as_retriever()
        
# Test d’une requête
print("🔍 Résultat de la requête test:")
test_question = "Quelles sont les actions les plus performantes en ce moment ?"
results_inner = retriever_inner.get_relevant_documents(test_question, k=1)   # la question est encodée en vecteur avec le même embedding
print(results_inner[0].page_content)

🔍 Résultat de la requête test:
Moreover, momentum stocks serve as a strategic tool for portfolio diversification. They offer exposure to various sectors and industries, potentially mitigating overall risk. The approach taps into the psychological aspect of investing, leveraging the human tendency to ‘follow the herd.’
With that said, these selections stand out for their robust performance and upward trajectories. They also inspire confidence among those who scrutinize the market’s every pulse. Consequently, each stock in this list represents a unique opportunity.
